<a href="https://colab.research.google.com/github/shahriarivari/Persian_sentiment_analysis/blob/main/BERT_notebooks/Pretrain_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pip installs

In [1]:
!pip install tokenizers
!pip install datasets
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


#import libraries

In [2]:
import os
from transformers import BertTokenizerFast
from transformers import BertModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import TrainerCallback
import logging

##import dataset

In [3]:
# You should just change this part in order to download your
# parts of corpus.
indices = {
    "train": [81, 14, 3,
              # 94, 35,
              # 41, 28, 67, 55, 79
              ],
    "test": [0,
            #  1
             ]
}

N_FILES = {
    "train": 126,
    "test": 3
}
_BASE_URL = "https://huggingface.co/datasets/SLPL/naab/resolve/main/data/"
data_url = {
    "train": [_BASE_URL + "train-{:05d}-of-{:05d}.txt".format(x, N_FILES["train"]) for x in range(N_FILES["train"])],
    "test": [_BASE_URL + "test-{:05d}-of-{:05d}.txt".format(x, N_FILES["test"]) for x in range(N_FILES["test"])],
}
for index in indices['train']:
    assert index < N_FILES['train']
for index in indices['test']:
    assert index < N_FILES['test']
data_files = {
    "train": [data_url['train'][i] for i in indices['train']],
    "test": [data_url['test'][i] for i in indices['test']]
}

In [5]:
dataset = load_dataset('text', data_files=data_files, use_auth_token=False)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

#set paths and file names

In [4]:
# Set your paths and file names
tokenizer_output_dir = "bert_tokenizer"
os.mkdir(tokenizer_output_dir)
pretrained_model_output_dir = "bert_pretrained_model"
os.mkdir(pretrained_model_output_dir)

##load tokenzier

In [6]:
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_output_dir)

##tokenize the dataset

In [8]:
def batch_tokenize_function(batch):
    # Tokenize the text using BertTokenizerFast
    encoded = tokenizer(batch["text"], truncation=True, padding='max_length',
                        return_special_tokens_mask=True,
                        max_length= 256, return_tensors='pt')

    return {
        'input_ids': encoded['input_ids'],
        'attention_mask': encoded['attention_mask'],
    }

tokenized_train_dataset = dataset['train'].select([i for i in range(1_000_000)]).map(batch_tokenize_function, batched=True)
tokenized_test_dataset = dataset['test'].select([i for i in range(1_000_000)]).map(batch_tokenize_function, batched=True)

# tokenized_train_dataset = dataset['train'].map(batch_tokenize_function, batched=True)
# tokenized_test_dataset = dataset['test'].map(batch_tokenize_function, batched=True)

tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
tokenized_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

##BERT model configuration

In [14]:
from transformers import BertConfig , BertForMaskedLM
vocab_size = 10000
max_length = 256

# Model Configuration
model_config = BertConfig(
    vocab_size=vocab_size,
    hidden_size=64,  # Adjust as needed
    num_attention_heads=8,  # Adjust as needed
    num_hidden_layers=4,  # Adjust as needed
    max_position_embeddings=max_length,
)

# Model Initialization
model = BertForMaskedLM(config=model_config)

##set the data collator

In [15]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
    )

##training arguments

In [16]:
# Training Arguments
training_args = TrainingArguments(
    output_dir=pretrained_model_output_dir, # output directory to where save model checkpoint
    overwrite_output_dir=True,
    num_train_epochs=1,                     # number of training epochs, feel free to tweak
    per_device_train_batch_size=64,         # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=100,        # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,          # evaluation batch size
    logging_steps=1000,                     # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    save_total_limit=3,                     # whether you don't have much space so you let only 3 model weights saved in the disk
    evaluation_strategy="steps",            # evaluate each `logging_steps` steps
    eval_steps=10000,
)

##train the model

In [17]:
# Trainer Initialization
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_test_dataset,
    data_collator = data_collator,
)

# Train the model
trainer.train()

# Save the final pre-trained model
trainer.save_model(os.path.join(pretrained_model_output_dir, "final_model"))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


##saving the model in google drive

In [ ]:
#First zip the model_dir then move it to google drive

source_dir = '/content/bert_pretrained_model'
output_filename = pretrained_model_output_dir

# Create a Zip file
shutil.make_archive(output_filename, 'zip', source_dir)

In [ ]:
# mount google  drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Move the zip file to Google Drive
drive_path = '/content/drive/MyDrive/'
output_filename = f'/content/{output_filename}.zip'
shutil.move(output_filename, drive_path)